In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
train_df = pd.read_csv("/Users/felipepesantez/Documents/development/datasets/svm/human_activity/train.csv")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 563 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), int64(1), object(1)
memory usage: 31.6+ MB


In [4]:
train_df.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515,17.413085
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122,8.975143
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414,8.000000
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181,19.000000
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659,26.000000
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000,30.000000


In [5]:
train_df.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity'],
      dtype='object', length=563)

In [6]:
train_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [7]:
train_df['Activity'].value_counts()

Activity
LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: count, dtype: int64

In [8]:
class_names = {"LAYING":0,
               "STANDING": 1,
               "SITTING": 2,
               "WALKING": 3,
               "WALKING_UPSTAIRS": 4,
               "WALKING_DOWNSTAIRS": 5}

In [9]:
X = train_df.drop('Activity',axis=1)
y = train_df['Activity']

In [10]:
y = train_df['Activity'].map(class_names)

In [11]:
y

0       1
1       1
2       1
3       1
4       1
       ..
7347    4
7348    4
7349    4
7350    4
7351    4
Name: Activity, Length: 7352, dtype: int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=10)

In [13]:
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [14]:
y_pred = svm_model.predict(X_test)

In [15]:
print(confusion_matrix(y_test, y_pred))
print("----------------")
print(classification_report(y_test, y_pred))

[[275   0   0   0   0   0]
 [  0 231  31   0   0   0]
 [  4  43 223   0   0   0]
 [  0   0   0 224   2   0]
 [  0   0   0   4 217   4]
 [  0   0   0   2   5 206]]
----------------
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       275
           1       0.84      0.88      0.86       262
           2       0.88      0.83      0.85       270
           3       0.97      0.99      0.98       226
           4       0.97      0.96      0.97       225
           5       0.98      0.97      0.97       213

    accuracy                           0.94      1471
   macro avg       0.94      0.94      0.94      1471
weighted avg       0.94      0.94      0.94      1471



GRID SEARCH

In [16]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}

In [17]:
grid_model = GridSearchCV(SVC(), param_grid, verbose=3)
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ....................C=0.1, gamma=1;, score=0.192 total time=  12.2s
[CV 2/5] END ....................C=0.1, gamma=1;, score=0.192 total time=  12.2s
[CV 3/5] END ....................C=0.1, gamma=1;, score=0.192 total time=  12.3s
[CV 4/5] END ....................C=0.1, gamma=1;, score=0.193 total time=  12.3s
[CV 5/5] END ....................C=0.1, gamma=1;, score=0.193 total time=  12.3s
[CV 1/5] END ..................C=0.1, gamma=0.1;, score=0.781 total time=  10.4s
[CV 2/5] END ..................C=0.1, gamma=0.1;, score=0.786 total time=  10.5s
[CV 3/5] END ..................C=0.1, gamma=0.1;, score=0.767 total time=  10.5s
[CV 4/5] END ..................C=0.1, gamma=0.1;, score=0.776 total time=  10.9s
[CV 5/5] END ..................C=0.1, gamma=0.1;, score=0.790 total time=  10.7s
[CV 1/5] END .................C=0.1, gamma=0.01;, score=0.904 total time=   4.7s
[CV 2/5] END .................C=0.1, gamma=0.01

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

In [18]:
grid_model.best_score_

0.9918378906362886

In [19]:
grid_model.best_params_

{'C': 100, 'gamma': 0.01}

In [20]:
grid_model.best_estimator_

SVC(C=100, gamma=0.01)

In [21]:
grid_preds = grid_model.predict(X_test)

In [22]:
print(confusion_matrix(y_test, grid_preds))
print("----------------")
print(classification_report(y_test, grid_preds))

[[275   0   0   0   0   0]
 [  0 254   8   0   0   0]
 [  0   5 265   0   0   0]
 [  0   0   0 225   1   0]
 [  0   0   0   0 225   0]
 [  0   0   0   0   0 213]]
----------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       275
           1       0.98      0.97      0.98       262
           2       0.97      0.98      0.98       270
           3       1.00      1.00      1.00       226
           4       1.00      1.00      1.00       225
           5       1.00      1.00      1.00       213

    accuracy                           0.99      1471
   macro avg       0.99      0.99      0.99      1471
weighted avg       0.99      0.99      0.99      1471



In [23]:
import pickle

with open("grid_model.pkl","wb") as file:
    pickle.dump(grid_model, file)

INFERENCE

In [24]:
test_df = pd.read_csv("/Users/felipepesantez/Documents/development/datasets/svm/human_activity/test.csv")

In [25]:
y_test_inference = test_df.drop('Activity',axis=1)

In [26]:
predictions = grid_model.predict(y_test_inference)

In [27]:
print(y_test_inference.iloc[0])

tBodyAcc-mean()-X                       0.257178
tBodyAcc-mean()-Y                      -0.023285
tBodyAcc-mean()-Z                      -0.014654
tBodyAcc-std()-X                       -0.938404
tBodyAcc-std()-Y                       -0.920091
                                          ...   
angle(tBodyGyroJerkMean,gravityMean)    0.271151
angle(X,gravityMean)                   -0.720009
angle(Y,gravityMean)                    0.276801
angle(Z,gravityMean)                   -0.057978
subject                                 2.000000
Name: 0, Length: 562, dtype: float64


In [28]:
predictions[0]

1

In [29]:
class_names

{'LAYING': 0,
 'STANDING': 1,
 'SITTING': 2,
 'WALKING': 3,
 'WALKING_UPSTAIRS': 4,
 'WALKING_DOWNSTAIRS': 5}

In [30]:
test_df['Activity'].iloc[0]

'STANDING'